#  Import Libraries

In [5]:
!pip install cma
# %pip install cma // to install in same environment as jupyter notebook

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, dual_annealing
import cma
from sklearn.datasets import fetch_openml
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, train_test_split


#  Define Benchmark Functions

In [7]:
# Rosenbrock Function
def rosenbrock(x):
    return sum(100.0*(x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

# Rastrigin Function
def rastrigin(x):
    return 10*len(x) + sum(x**2 - 10*np.cos(2*np.pi*x))

# Ackley Function
def ackley(x):
    first_sum = np.sum(x**2)
    second_sum = np.sum(np.cos(2*np.pi*x))
    n = len(x)
    return -20*np.exp(-0.2*np.sqrt(first_sum/n)) - np.exp(second_sum/n) + 20 + np.e

# Optimization on Benchmark Functions (Task 1)

In [8]:
methods = ['Nelder-Mead', 'Simulated Annealing', 'CMA-ES']
functions = {'Rosenbrock': rosenbrock, 'Rastrigin': rastrigin, 'Ackley': ackley}
bounds = [(-5,5), (-5,5)]
initial_guess = [3,3]

results = {}

for fname, func in functions.items():
    results[fname] = {}
    
    # Nelder-Mead
    res_nm = minimize(func, initial_guess, method='Nelder-Mead')
    
    # Simulated Annealing
    res_sa = dual_annealing(func, bounds=bounds)
    
    # CMA-ES
    res_cma = cma.fmin(func, initial_guess, sigma0=0.5, options={'verb_disp': False})
    
    results[fname]['Nelder-Mead'] = res_nm.fun
    results[fname]['Simulated Annealing'] = res_sa.fun
    results[fname]['CMA-ES'] = res_cma[1]

print("Benchmark Results:")
for fname in results:
    print(f"\n{fname} function:")
    for method in methods:
        print(f"{method}: {results[fname][method]}")

Benchmark Results:

Rosenbrock function:
Nelder-Mead: 1.0423182566004013e-09
Simulated Annealing: 8.888116134953508e-12
CMA-ES: 1.1518594611976732e-17

Rastrigin function:
Nelder-Mead: 17.90920266864526
Simulated Annealing: 2.842170943040401e-14
CMA-ES: 0.2419253812634281

Ackley function:
Nelder-Mead: 9.00109350489619
Simulated Annealing: 1.7545432218213364e-08
CMA-ES: 9.001093478292006


# Load MNIST Dataset & Preprocess (Task 2)

In [1]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data[:2000], mnist.target[:2000] # subset for speed

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

NameError: name 'fetch_openml' is not defined

#  Define Hyperparameter Optimization Function (SVM)

In [10]:
def svm_eval(params):
    C, gamma = params[0], params[1]
    clf = SVC(C=C, gamma=gamma)
    scores = cross_val_score(clf, X_train, y_train, cv=3)
    return 1 - scores.mean() # minimize error rate

#  Hyperparameter Tuning with Optimization Methods (Task 2)

In [ ]:
hp_bounds = [(0.1,100), (1e-4,1)]

# Nelder-Mead Optimization
res_svm_nm = minimize(svm_eval, x0=[1.0, 0.01], method='Nelder-Mead')

# Simulated Annealing Optimization
res_svm_sa = dual_annealing(svm_eval, bounds=hp_bounds)

# CMA-ES Optimization
res_svm_cma = cma.fmin(svm_eval, [1.0, 0.01], sigma0=0.5, options={'bounds': [np.array([0.1,1e-4]), np.array([100,1])]})

print("\nSVM Hyperparameter Tuning Results:")
print("Nelder-Mead optimal params:", res_svm_nm.x)
print("Simulated Annealing optimal params:", res_svm_sa.x)
print("CMA-ES optimal params:", res_svm_cma[0])


# Evaluate Final Test Accuracy (Task 2 Continued)

In [ ]:
def test_accuracy(params):
    C_opt,gamma_opt=params[0],params[1]
    clf=SVC(C=C_opt,gamma=gamma_opt).fit(X_train,y_train)
    return clf.score(X_test,y_test)

acc_nm=test_accuracy(res_svm_nm.x)
acc_sa=test_accuracy(res_svm_sa.x)
acc_cma=test_accuracy(res_svm_cma[0])

print("\nFinal Test Accuracy:")
print(f"Nelder-Mead Accuracy: {acc_nm:.4f}")
print(f"Simulated Annealing Accuracy: {acc_sa:.4f}")
print(f"CMA-ES Accuracy: {acc_cma:.4f}")

#  Visualization of Optimization Trajectories (Task 3 Example with Rosenbrock)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

x=np.linspace(-3,3,100)
y=np.linspace(-3,3,100)
X,Y=np.meshgrid(x,y)
Z=(1-X)**2+100*(Y-X**2)**2

fig=plt.figure(figsize=(10,7))
ax=fig.add_subplot(111,projection='3d')
ax.plot_surface(X,Y,Z,cmap='viridis',alpha=0.8)

ax.set_title('Rosenbrock Function')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('f(X,Y)')
plt.show()

# Hyperparameter Landscape Visualization (Task 3 Example for SVM)

In [ ]:
C_vals=np.logspace(-1,2,num=10)
gamma_vals=np.logspace(-4,-1,num=10)

accuracy_grid=np.zeros((len(C_vals),len(gamma_vals)))

for i,C in enumerate(C_vals):
    for j,gamma in enumerate(gamma_vals):
        clf=SVC(C=C,gamma=gamma).fit(X_train,y_train)
        accuracy_grid[i,j]=clf.score(X_test,y_test)

plt.figure(figsize=(8,6))
plt.imshow(accuracy_grid,cmap='viridis',origin='lower')
plt.colorbar(label='Accuracy')
plt.xticks(range(len(gamma_vals)),np.round(gamma_vals,4),rotation=45)
plt.yticks(range(len(C_vals)),np.round(C_vals,3))
plt.xlabel('Gamma')
plt.ylabel('C')
plt.title('SVM Hyperparameter Landscape')
plt.show()
